In [48]:
import os
import json
import argparse
from tqdm import tqdm


In [49]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='BDD100K to COCO format')
    parser.add_argument(
           "-l", "--label_path",
           default="./data/bdd/labels",
           help="root directory of BDD label Json files",
    )
    parser.add_argument(
          "-s", "--save_path",
          default="./convert_data/bdd2coco",
          help="path to save coco formatted label file",
    )
    return parser.parse_args()


In [50]:
def bdd2coco_detection(id_dict, labeled_images, fn):
    images = list()
    annotations = list()
    
    counter = 0
    for i in tqdm(labeled_images):
        counter += 1
        image = dict()
        image['file_name'] = i['name']
        image['height'] = 720
        image['width'] = 1280
        
        image['id'] = counter
        
        empty_image = True
        
        for l in i['labels']:
            annotation = dict()
            if l['category'] in id_dict.keys():
                empty_image = False
                annotation["iscrowd"] = 0
                annotation["image_id"] = image['id']
                x1 = l['box2d']['x1']
                y1 = l['box2d']['y1']
                x2 = l['box2d']['x2']
                y2 = l['box2d']['y2']
                annotation['bbox'] = [x1, y1, x2-x1,y2-y1]
                annotation['area'] = float((x2-x1)*(y2-y1))
                annotation['category_id'] = id_dict[l['category']]
                annotation['ignore'] = 0
                annotation['id'] = l['id']
                annotation['segmentation'] = [[x1,y1,x1,y2,x2,y2,x2,y1]]
                annotations.append(annotation)
        if empty_image:
            continue
        
        images.append(image)
        
    attr_dict["images"] = images
    attr_dict["annotations"] = annotations
    attr_dict["type"] = "instances"
    
    print('saving...')
    json_string = json.dumps(attr_dict)
    with open(fn, "w") as file:
        file.write(json_string)
        

            

In [57]:
        
if __name__ == '__main__':
    #args = parse_arguments()
    
    label_dir = './data/bdd/labels'
    save_path = './convert_data/bdd2coco'
    
    attr_dict = dict()
    
    attr_dict["categories"] = [
        {"supercategory":"none","id":1,"name":"person"},
        {"supercategory":"none","id":2,"name":"rider"},
        {"supercategory":"none","id":3,"name":"car"},
        {"supercategory":"none","id":4,"name":"bus"},
        {"supercategory":"none","id":5,"name":"truck"},
        {"supercategory":"none","id":6,"name":"bike"},
        {"supercategory":"none","id":7,"name":"motor"},
        {"supercategory":"none","id":8,"name":"traffic light"},
        {"supercategory":"none","id":9,"name":"traffic sign"},
        {"supercategory":"none","id":10,"name":"train"},
    ]
    
    attr_id_dict = {i['name']:i['id'] for i in attr_dict['categories']}
        
    
        

In [58]:
    # create BDD training set detections in COCO format
    print('Loading training set...')
    with open(os.path.join(label_dir,'bdd100k_labels_images_train.json')) as f:     
        train_labels = json.load(f)
    print('Converting training set...')
    
    out_fn = os.path.join(save_path,
                         'bdd100k_labels_images_det_coco_train.json')
    bdd2coco_detection(attr_id_dict, train_labels,out_fn)
    
 
    

Loading training set...


  3%|▎         | 2094/69863 [00:00<00:03, 20936.08it/s]

Converting training set...


100%|██████████| 69863/69863 [00:07<00:00, 9411.06it/s] 


saving...


In [59]:
    # create BDD validation set detection in COCO format
    print('Loading validation set...')
    with open(os.path.join(label_dir,'bdd100k_labels_images_val.json')) as f:
        val_labels = json.load(f)
    print('Converting validation set...')
    
    out_fn = os.path.join(save_path,
                         'bdd100k_labels_images_det_coco_val.json')
    bdd2coco_detection(attr_id_dict, val_labels, out_fn)

Loading validation set...


 18%|█▊        | 1808/10000 [00:00<00:00, 18079.72it/s]

Converting validation set...


100%|██████████| 10000/10000 [00:00<00:00, 18674.80it/s]


saving...
